<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/TEST%20Lukas%20Zumthurm%20Verpressanker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-IFC-Dashboard-Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IFC Dashboard Starter Template

A minimal template to get started building IFC data dashboards.

**What this does:**
- Load an IFC file from GitHub
- Extract element types and building storeys
- Create a simple 2-tab dashboard


## Step 1: Install packages


In [102]:
%pip install -q ifcopenshell pandas plotly dash xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 6.3 MB/s eta 0:00:00


## Step 2: Load IFC file from GitHub


In [93]:
import urllib.request
import ifcopenshell

# Raw IFC file URL from GitHub
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-D/main/TEST%20LZ/IFC-PA2-HS2526%20Baugrubenabschluss%2011.11.2025.ifc'
local_filename = 'IFC-PA2-HS2526 Baugrubenabschluss 11.11.2025.ifc'

print("📥 Downloading IFC file...")
urllib.request.urlretrieve(github_url, local_filename)
print("✅ Downloaded successfully!")

# Open IFC file
model = ifcopenshell.open(local_filename)
print(f"\n🏗️ Loaded: {model.schema}")
print(f"📊 Total elements: {len(model.by_type('IfcElement'))}")



📥 Downloading IFC file...
✅ Downloaded successfully!

🏗️ Loaded: IFC2X3
📊 Total elements: 321


## Step 3: Parse IFC data

Extract elements and their building storeys.


In [96]:
import pandas as pd

# Helper function to get storey name
def get_storey_name(storey):
    if storey is None:
        return 'Not assigned'
    return storey.LongName or storey.Name or f"Storey {storey.id()}"

# Helper function to get property from a PSet
def get_pset_property(element, pset_name, prop_name):
    for definition in element.IsDefinedBy:
        if definition.is_a('IfcRelDefinesByProperties'):
            prop_set = definition.RelatingPropertyDefinition
            if prop_set.is_a('IfcPropertySet') and prop_set.Name == pset_name:
                for prop in prop_set.HasProperties:
                    if prop.Name == prop_name:
                        return getattr(prop, 'NominalValue', getattr(prop, 'Value', None))
    return None

# Collect only Verpressanker elements
element_data = []

for element in model.by_type('IfcElement'):
    # Check if element has Pset_Verpressanker
    has_verpressanker = any(
        definition.is_a('IfcRelDefinesByProperties') and
        definition.RelatingPropertyDefinition.is_a('IfcPropertySet') and
        definition.RelatingPropertyDefinition.Name == 'Pset_Verpressanker'
        for definition in element.IsDefinedBy
    )
    if not has_verpressanker:
        continue

    # Basic info
    element_id = element.GlobalId
    element_type = get_pset_property(element, 'Pset_Verpressanker', 'Typ')
    bore_depth = get_pset_property(element, 'Pset_Verpressanker', 'Bohrtiefe')

    # Find storey
    storey = 'Not assigned'
    for rel in model.by_type('IfcRelContainedInSpatialStructure'):
        if element in rel.RelatedElements:
            if rel.RelatingStructure and rel.RelatingStructure.is_a('IfcBuildingStorey'):
                storey = get_storey_name(rel.RelatingStructure)
                break

    element_data.append({
        'ElementId': element_id,
        'Storey': storey,
        'Typ': element_type,
        'Bohrtiefe': bore_depth
    })

# Create DataFrame
df = pd.DataFrame(element_data)

print(f"✅ Parsed {len(df)} Verpressanker records")
df.head()


✅ Parsed 158 Verpressanker records


,ElementId,Storey,Typ,Bohrtiefe
0,1S7Tl2aML3OuxrctBXYpK$,Aushubsohle,"[KSB Standard B 500 R32/22, KSB Bohrkronentyp...",[4.999999999999999]
1,1S7Tl2aML3OuxrctBXYpQ2,Aushubsohle,"[KSB Standard B 500 R38/17, KSB Bohrkronentyp ...",[5.499999999999999]
2,0fhMkCxOP0VOgq$ymka6oR,Aushubsohle,"[KSB Standard B 500 R32/17, KSB Bohrkronentyp ...",[4.999999999999999]
3,0fhMkCxOP0VOgq$ymka6pc,2.UG,"[KSB Standard B 500 R32/22, KSB Bohrkronentyp ...",[5.299999999999999]
4,0qqk4rnb5BXfvvOn0JsGe7,Aushubsohle,"[KSB Standard B 500 R32/22, KSB Bohrkronentyp...",[4.999999999999999]


## Step 4: Create aggregations for visualization


In [98]:
import pandas as pd

# Helper function to get storey name
def get_storey_name(storey):
    if storey is None:
        return 'Not assigned'
    return storey.LongName or storey.Name or f"Storey {storey.id()}"

# Helper function to extract all properties from a PSet
def get_pset_properties(element, pset_name):
    props = {}
    for definition in element.IsDefinedBy:
        if definition.is_a('IfcRelDefinesByProperties'):
            prop_set = definition.RelatingPropertyDefinition
            if prop_set.is_a('IfcPropertySet') and prop_set.Name == pset_name:
                for prop in prop_set.HasProperties:
                    # Handle numeric IFC values
                    val = getattr(prop, 'NominalValue', getattr(prop, 'Value', None))
                    if hasattr(val, 'wrappedValue'):  # If IFC wrapper
                        val = val.wrappedValue
                    props[prop.Name] = val
    return props

# Collect Verpressanker elements
element_data = []

for element in model.by_type('IfcElement'):
    # Check if element has Pset_Verpressanker
    has_verpressanker = any(
        definition.is_a('IfcRelDefinesByProperties') and
        definition.RelatingPropertyDefinition.is_a('IfcPropertySet') and
        definition.RelatingPropertyDefinition.Name == 'Pset_Verpressanker'
        for definition in element.IsDefinedBy
    )
    if not has_verpressanker:
        continue

    # Find storey
    storey = 'Not assigned'
    for rel in model.by_type('IfcRelContainedInSpatialStructure'):
        if element in rel.RelatedElements:
            if rel.RelatingStructure and rel.RelatingStructure.is_a('IfcBuildingStorey'):
                storey = get_storey_name(rel.RelatingStructure)
                break

    # Extract all properties from Pset_Verpressanker
    pset_props = get_pset_properties(element, 'Pset_Verpressanker')

    # Build record
    record = {'ElementId': element.GlobalId, 'Storey': storey}
    record.update(pset_props)
    element_data.append(record)

# Create DataFrame
df = pd.DataFrame(element_data)

# Optional: convert numeric properties to float if possible
for col in df.columns:
    if col not in ['ElementId', 'Storey']:
        df[col] = pd.to_numeric(df[col], errors='ignore')

print(f"✅ Parsed {len(df)} Verpressanker records with dynamic PSet properties")
df.head()



✅ Parsed 158 Verpressanker records with dynamic PSet properties


/tmp/ipython-input-525879412.py:60: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



,ElementId,Storey,Typ,Bohrtiefe
0,1S7Tl2aML3OuxrctBXYpK$,Aushubsohle,"KSB Standard B 500 R32/22, KSB Bohrkronentyp ...",5.0
1,1S7Tl2aML3OuxrctBXYpQ2,Aushubsohle,"KSB Standard B 500 R38/17, KSB Bohrkronentyp S...",5.5
2,0fhMkCxOP0VOgq$ymka6oR,Aushubsohle,"KSB Standard B 500 R32/17, KSB Bohrkronentyp S...",5.0
3,0fhMkCxOP0VOgq$ymka6pc,2.UG,"KSB Standard B 500 R32/22, KSB Bohrkronentyp S...",5.3
4,0qqk4rnb5BXfvvOn0JsGe7,Aushubsohle,"KSB Standard B 500 R32/22, KSB Bohrkronentyp ...",5.0


## Step 5: Build a simple dashboard


In [105]:
# --- Diagramme ---
type_chart = px.bar(
    type_counts,
    x='Typ',
    y='Count',
    title='Anzahl Verpressanker pro Typ',
    text_auto='.0f'
)
type_chart.update_layout(xaxis_title='Verpressanker Typ', yaxis_title='Anzahl', xaxis_tickangle=-45)

bohr_chart = px.bar(
    bohrtiefe_sum,
    x='Typ',
    y='Total_Bohrtiefe',
    title='Totale Bohrlänge pro Typ',
    text_auto='.2f'
)
bohr_chart.update_layout(xaxis_title='Verpressanker Typ', yaxis_title='Totale Bohrlänge [mm]', xaxis_tickangle=-45)

# --- Dash App ---
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Verpressanker Analyse Dashboard'),
    html.H3(f'Gesamte Bohrlänge: {total_bohrtiefe:.2f} mm'),

    html.A(
        "📥 Excel Export",
        id='download-link',
        download="Verpressanker.xlsx",
        href=generate_excel_download_link(df),
        target="_blank",
        style={'marginBottom': '20px', 'display': 'block'}
    ),

    dcc.Tabs([
        dcc.Tab(label='Anzahl pro Typ', children=[
            dcc.Graph(figure=type_chart)
        ]),
        dcc.Tab(label='Totale Bohrlänge pro Typ', children=[
            dcc.Graph(figure=bohr_chart)
        ]),
        dcc.Tab(label='Rohdaten', children=[
            dash_table.DataTable(
                df.to_dict('records'),
                page_size=15,
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left'}
            )
        ])
    ])
])

print("🚀 Starting Verpressanker Dashboard...")
app.run(jupyter_mode='inline', height=800, port=8050)

🚀 Starting Verpressanker Dashboard...


<IPython.core.display.Javascript object>

## Next Steps

**Extend this template:**
- Add more tabs for different analyses
- Include materials data
- Add filters with dropdowns
- Try different chart types

**Try other files:**
- Change the `github_url` to load different IFC files
- Upload your own IFC files using file upload widgets

**Learn more:**
- [IFCOpenShell Documentation](https://docs.ifcopenshell.org/)
- [Plotly Dash Documentation](https://dash.plotly.com/)
- [Full example: BFH-25-Tabbed-Dashboard.ipynb](https://github.com/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-Tabbed-Dashboard.ipynb)
